# Data Mapping: Enrollment and Demographics with Crosswalks

## Loading excel sheets into dataframe



In [0]:
%python
# pip install openpyxl


In [0]:
%python

from pyspark.sql import SparkSession
from pyspark.sql.types import StringType, StructType, StructField

In [0]:
%python
# Initialize Spark session
spark = SparkSession.builder \
    .appName("Excel to Spark DataFrame") \
    .getOrCreate()

In [0]:
%python
# Define the file location
file_location = "/FileStore/shared_uploads/sushantpandeyofficial@gmail.com/Eligibility_Mock_Data___US_Healthcare_Bootcamp.xlsx"
sheet_name = "Enrollment Mock Data"

# Load the Excel file into a Spark DataFrame
df = spark.read.format("com.crealytics.spark.excel") \
    .option("header", True) \
    .option("inferSchema", True) \
    .option("dataAddress", f"'{sheet_name}'!") \
    .load(file_location)

# Show the DataFrame schema
df.printSchema()

# Register the DataFrame as a temporary view
df.createOrReplaceTempView("enrollment_mock_data")

# Query the DataFrame using SQL
spark.sql("SELECT * FROM enrollment_mock_data").show()

# # Stop the Spark session
# spark.stop()


root
 |-- MEMBER_ID: string (nullable = true)
 |-- MEMBER_STATUS: string (nullable = true)
 |-- GROUP_ID: string (nullable = true)
 |-- COVERAGE_TYPE: string (nullable = true)
 |-- PLAN_ID: double (nullable = true)
 |-- VENDOR: string (nullable = true)
 |-- EFFECTIVE_DATE: timestamp (nullable = true)
 |-- TERMINATION_DATE: string (nullable = true)

+---------+-------------+--------+-------------+-------+------+-------------------+----------------+
|MEMBER_ID|MEMBER_STATUS|GROUP_ID|COVERAGE_TYPE|PLAN_ID|VENDOR|     EFFECTIVE_DATE|TERMINATION_DATE|
+---------+-------------+--------+-------------+-------+------+-------------------+----------------+
|   173400|       ACTIVE|       1|          E1C|   11.0| Cigna|2021-01-01 00:00:00|      2021-02-28|
|    83300|       ACTIVE|       1|           ES|    0.0| Cigna|2019-12-01 00:00:00|      2020-01-31|
|   106800|       ACTIVE|       1|            U|    4.0| Aetna|2020-09-01 00:00:00|      2020-10-31|
|    52900|       ACTIVE|       1|         

In [0]:
%python
sheet_name2 = "Demographics Mock Data"

# Load the Excel file into a Spark DataFrame
df = spark.read.format("com.crealytics.spark.excel") \
    .option("header", True) \
    .option("inferSchema", True) \
    .option("dataAddress", f"'{sheet_name2}'!") \
    .load(file_location)

# Show the DataFrame schema
df.printSchema()

# Register the DataFrame as a temporary view
df.createOrReplaceTempView("demographics_mock_data")

# Query the DataFrame using SQL
spark.sql("SELECT * FROM demographics_mock_data").show()

# # Stop the Spark session
# spark.stop()

root
 |-- MEMBER_ID: string (nullable = true)
 |-- EMPLOYEE_ID: string (nullable = true)
 |-- FIRST_NAME: string (nullable = true)
 |-- LAST_NAME: string (nullable = true)
 |-- MIDDLE_NAME: string (nullable = true)
 |-- GENDER: double (nullable = true)
 |-- DOB: string (nullable = true)
 |-- RELATIONSHIP: string (nullable = true)
 |-- PERSON_CODE: string (nullable = true)
 |-- RACE: string (nullable = true)
 |-- ETHNICITY: string (nullable = true)
 |-- ADDRESS_1: string (nullable = true)
 |-- ADDRESS_2: string (nullable = true)
 |-- CITY: string (nullable = true)
 |-- STATE: string (nullable = true)
 |-- COUNTY: string (nullable = true)
 |-- ZIP: string (nullable = true)

+---------+-----------+----------+----------+-----------+------+----------+------------+-----------+-----+-----------+--------------------+---------+------------+-------------+----------------+-----+
|MEMBER_ID|EMPLOYEE_ID|FIRST_NAME| LAST_NAME|MIDDLE_NAME|GENDER|       DOB|RELATIONSHIP|PERSON_CODE| RACE|  ETHNICITY| 

## Creating tables from crosswalk sheet

In [0]:
%python
import pandas as pd

# Define the table ranges and sheet name
table_ranges = ["A2:C5", "A8:B25", "A28:B33", "A36:B43", "A46:E62"]
sheet_name = "crosswalks"

# Initialize a list to store table creation results
table_results = []

# Iterate over the table ranges
for idx, table_range in enumerate(table_ranges, start=1):
    try:
        # Read data for the specified table range
        df = spark.read.format("com.crealytics.spark.excel") \
            .option("inferschema", True) \
            .option("header", True) \
            .option("dataAddress", f"{sheet_name}!{table_range}") \
            .option("sheetName", sheet_name) \
            .load(file_location)

        # Generate table name and create temporary view
        processed_sheet_name = sheet_name.lower().replace(" ", "_").replace("'", "")
        table_name = f"{processed_sheet_name}_Table{idx}"
        df.createOrReplaceTempView(table_name)

        # Append table creation result to the list
        table_results.append({"Table Range": table_range, "Table Name": table_name, "Status": "Success"})
    except AnalysisException as e:
        # Append error message if any exception occurs
        table_results.append({"Table Range": table_range, "Table Name": "N/A", "Status": f"Error: {str(e)}"})

# Convert the list of dictionaries to a DataFrame
table_results_df = pd.DataFrame(table_results)

# Display the table results
print("Table Creation Results:")
print(table_results_df)


Table Creation Results:
  Table Range         Table Name   Status
0       A2:C5  crosswalks_Table1  Success
1      A8:B25  crosswalks_Table2  Success
2     A28:B33  crosswalks_Table3  Success
3     A36:B43  crosswalks_Table4  Success
4     A46:E62  crosswalks_Table5  Success


## Target table

In [0]:
%sql
CREATE TABLE IF NOT EXISTS TargetTable (
    Abacus_Record_ID VARCHAR(20),
    Abacus_Member_ID VARCHAR(20),
    Member_ID VARCHAR(20),
    Subscriber_ID VARCHAR(20),
    Member_First_Name VARCHAR(75),
    Member_Last_Name VARCHAR(75),
    Member_Middle_Name VARCHAR(75),
    Member_Prefix_Name VARCHAR(10),
    Member_Suffix_Name VARCHAR(10),
    Member_Gender VARCHAR(10),
    Member_Date_of_Birth DATE,
    Member_Relationship_Code VARCHAR(10),
    Member_Person_Code INTEGER,
    Member_Address_Line_1 VARCHAR(100),
    Member_Address_Line_2 VARCHAR(100),
    Member_City VARCHAR(20),
    Member_State VARCHAR(20),
    Member_County VARCHAR(50),
    Member_Postal_Code VARCHAR(10),
    Member_Country VARCHAR(20),
    Member_Home_Phone INTEGER,
    Member_Work_Phone INTEGER,
    Member_Mobile_Phone INTEGER,
    Member_Email VARCHAR(100),
    Member_Is_Deceased VARCHAR(10),
    Member_Date_of_Death DATE,
    Member_Deceased_Reason VARCHAR(100),
    Enrollment_Group_ID VARCHAR(20),
    Enrollment_Group_Name VARCHAR(50),
    Enrollment_SubGroup_ID VARCHAR(20),
    Enrollment_SubGroup_Name VARCHAR(50),
    Enrollment_Coverage_Code VARCHAR(10),
    Enrollment_Coverage_Description VARCHAR(30),
    Enrollment_Plan_ID VARCHAR(10),
    Enrollment_Plan_Name VARCHAR(30),
    Enrollment_Plan_Coverage VARCHAR(50),
    Enrollment_Medical_Effective_Date DATE,
    Enrollment_Medical_Termination_Date DATE,
    Enrollment_Dental_Effective_Date DATE,
    Enrollment_Dental_Termination_Date DATE,
    Enrollment_Vision_Effective_Date DATE,
    Enrollment_Vision_Termination_Date DATE,
    Enrollment_Vendor_Name VARCHAR(20),
    Souce_File_Name VARCHAR(100),
    File_Ingestion_Date DATE
);


In [0]:
%python
spark.conf.set("spark.sql.ansi.enabled", "false")


## Data Mapping into TargetTable

In [0]:
%sql
WITH demographics AS (
  SELECT
    D.*,
    ROW_NUMBER() OVER (ORDER BY D.member_id) AS Abacus_Record_Id,
    LEFT(CONCAT(E.member_id, '-', DATE_FORMAT(TO_DATE(D.dob), 'dyyyyM'), '-', SUBSTR(E.member_status, 1, 1), '-', E.member_id), 20) AS Abacus_Member_Id,
    COALESCE(D.middle_name, 'None') AS Member_Middle_Name,
    'prefix' AS Member_Prefix_Name,
    'suffix' AS Member_Suffix_Name,
    Gen.Rollup_Description AS Member_Gender,
    TRY_CAST(DATE_FORMAT(D.dob, 'MM/dd/yyyy') AS DATE) AS Member_Date_of_Birth,  -- Changed here
    D.relationship AS Member_Relationship_Code,
    D.person_code AS Member_Person_Code,
    D.address_1 AS Member_Address_Line_1,
    D.address_2 AS Member_Address_Line_2,
    D.city AS Member_City,
    D.state AS Member_State,
    D.county AS Member_County,
    D.zip AS Member_Postal_Code,
    'U.S.A' AS Member_Country,
    000 AS Member_Home_Phone,
    000 AS Member_Work_Phone,
    000 AS Member_Mobile_Phone,
    'member@gmail.com' AS Member_Email,
    'None' AS Member_Is_Deceased, 
    NULL AS Member_Date_of_Death,
    'None' AS Member_Deceased_Reason
  FROM demographics_mock_data D
  LEFT JOIN enrollment_mock_data E ON E.member_id = D.member_id
  JOIN crosswalks_table1 Gen ON Gen.code = D.gender
),
enrollment AS (
  SELECT 
    E.member_id AS Member_Id,
    E.member_status AS Subscriber_id,
    E.group_id AS Enrollment_Group_ID,
    Grp.group_name AS Enrollment_Group_Name,
    E.coverage_type AS Enrollment_Coverage_Code,
    Cov.coverage_description AS Enrollment_Coverage_Description,
    E.plan_id AS Enrollment_Plan_ID,
    Pln.`plan name` AS Enrollment_Plan_Name,
    Pln.benefit_type AS Enrollment_Plan_Coverage,
    TRY_CAST(Pln.effective_date AS DATE) AS Enrollment_Medical_Effective_Date,
    TRY_CAST(Pln.termination_date AS DATE) AS Enrollment_Medical_Termination_Date,
    E.vendor AS Enrollment_Vendor_Name
  FROM enrollment_mock_data E
  JOIN crosswalks_table3 Grp ON Grp.group_id = E.group_id
  JOIN crosswalks_table4 Cov ON Cov.coverage_ID = E.coverage_type
  JOIN crosswalks_table5 Pln ON Pln.plan_id = E.plan_id
)
INSERT INTO TargetTable
SELECT
    d.Abacus_Record_Id,
    d.Abacus_Member_Id,
    d.member_id AS Member_Id,
    e.Subscriber_id,
    d.first_name AS Member_First_Name,
    d.last_name AS Member_Last_Name, 
    d.Member_Middle_Name,
    d.Member_Prefix_Name,
    d.Member_Suffix_Name,
    d.Member_Gender,
    d.Member_Date_of_Birth,
    d.Member_Relationship_Code,
    d.Member_Person_Code,
    d.Member_Address_Line_1,
    d.Member_Address_Line_2,
    d.Member_City,
    d.Member_State,
    d.Member_County,
    d.Member_Postal_Code,
    d.Member_Country,
    d.Member_Home_Phone,
    d.Member_Work_Phone,
    d.Member_Mobile_Phone,
    d.Member_Email,
    d.Member_Is_Deceased, 
    d.Member_Date_of_Death,
    d.Member_Deceased_Reason,
    e.Enrollment_Group_ID,
    e.Enrollment_Group_Name,
    'None' AS Enrollment_SubGroup_ID,
    'None' AS Enrollment_SubGroup_Name,
    e.Enrollment_Coverage_Code,
    e.Enrollment_Coverage_Description,
    e.Enrollment_Plan_ID,
    e.Enrollment_Plan_Name,
    e.Enrollment_Plan_Coverage,
    e.Enrollment_Medical_Effective_Date,
    e.Enrollment_Medical_Termination_Date,
    NULL AS Enrollment_Dental_Effective_Date,
    NULL AS Enrollment_Dental_Termination_Date,
    NULL AS Enrollment_Vision_Effective_Date,
    NULL AS Enrollment_Vision_Termination_Date,
    e.Enrollment_Vendor_Name,
    'Member Enrolment' AS Source_File_Name,
    CURRENT_TIMESTAMP() AS File_Ingestion_Date
FROM demographics d
LEFT JOIN enrollment e ON e.Member_Id = d.Member_Id;

num_affected_rows,num_inserted_rows
129,129


In [0]:
%sql
select * from TargetTable;

Abacus_Record_ID,Abacus_Member_ID,Member_ID,Subscriber_ID,Member_First_Name,Member_Last_Name,Member_Middle_Name,Member_Prefix_Name,Member_Suffix_Name,Member_Gender,Member_Date_of_Birth,Member_Relationship_Code,Member_Person_Code,Member_Address_Line_1,Member_Address_Line_2,Member_City,Member_State,Member_County,Member_Postal_Code,Member_Country,Member_Home_Phone,Member_Work_Phone,Member_Mobile_Phone,Member_Email,Member_Is_Deceased,Member_Date_of_Death,Member_Deceased_Reason,Enrollment_Group_ID,Enrollment_Group_Name,Enrollment_SubGroup_ID,Enrollment_SubGroup_Name,Enrollment_Coverage_Code,Enrollment_Coverage_Description,Enrollment_Plan_ID,Enrollment_Plan_Name,Enrollment_Plan_Coverage,Enrollment_Medical_Effective_Date,Enrollment_Medical_Termination_Date,Enrollment_Dental_Effective_Date,Enrollment_Dental_Termination_Date,Enrollment_Vision_Effective_Date,Enrollment_Vision_Termination_Date,Enrollment_Vendor_Name,Souce_File_Name,File_Ingestion_Date
1,100000-14199010-A-10,100000,ACTIVE,Fergus,Matthews,None,prefix,suffix,Male,null,A2,12,503 Carter Light Apt 12,null,Auburn,Massachusetts,Worcester County,,U.S.A,0,0,0,member@gmail.com,None,null,None,2,Google,None,None,E1C,Employee and 1 Child,7.0,Plan H,"Medical, Dental and Vision",2019-01-01,2019-12-31,null,null,null,null,Cigna,Member Enrolment,2024-03-10
2,102200-3019949-A-102,102200,ACTIVE,Mohammad,Cox,None,prefix,suffix,Male,null,F,3,755 Hirthe Underpass,null,Milford,Massachusetts,Worcester County,,U.S.A,0,0,0,member@gmail.com,None,null,None,1,Abacus Insights,None,None,E1C,Employee and 1 Child,5.0,Plan F,Medical and Dental,2019-01-01,2019-12-31,null,null,null,null,Aetna,Member Enrolment,2024-03-10
3,105800-12199912-A-10,105800,ACTIVE,Erin,Huff,Kester,prefix,suffix,Male,null,G2,8,147 Rippin Lane,null,Boston,Massachusetts,Suffolk County,02116,U.S.A,0,0,0,member@gmail.com,None,null,None,1,Abacus Insights,None,None,U,Unknown,3.0,Plan D,"Medical, Dental and Vision",2018-01-01,2018-12-31,null,null,null,null,Aetna,Member Enrolment,2024-03-10
4,106800-2820155-A-106,106800,ACTIVE,Leanne,Swanson,None,prefix,suffix,Female,null,D2,16,686 Shields Dam,null,Easthampton,Massachusetts,Hampshire County,,U.S.A,0,0,0,member@gmail.com,None,null,None,1,Abacus Insights,None,None,U,Unknown,4.0,Plan E,Medical,2019-01-01,2019-12-31,null,null,null,null,Aetna,Member Enrolment,2024-03-10
5,106900-1819704-A-106,106900,ACTIVE,Stella,Franklin,None,prefix,suffix,Female,null,G1,7,818 Bogan Ville,null,Westport,Massachusetts,Bristol County,,U.S.A,0,0,0,member@gmail.com,None,null,None,8,TechKraft Inc,None,None,U,Unknown,12.0,Plan M,Medical,2021-01-01,2021-12-31,null,null,null,null,Cigna,Member Enrolment,2024-03-10
6,109600-2820083-A-109,109600,ACTIVE,Cole,Turner,None,prefix,suffix,Male,null,N,13,454 Crooks Divide Unit 13,null,Plymouth,Massachusetts,Plymouth County,,U.S.A,0,0,0,member@gmail.com,None,null,None,1,Abacus Insights,None,None,F,Family,4.0,Plan E,Medical,2019-01-01,2019-12-31,null,null,null,null,Cigna,Member Enrolment,2024-03-10
7,110800-2019979-T-110,110800,TERMED,Hollie,Eaton,None,prefix,suffix,Female,null,M,4,652 Kovacek Trailer Suite 29,null,Lenox,Massachusetts,Berkshire County,01240,U.S.A,0,0,0,member@gmail.com,None,null,None,8,TechKraft Inc,None,None,E1C,Employee and 1 Child,8.0,Plan I,Medical,2020-01-01,2020-12-31,null,null,null,null,Cigna,Member Enrolment,2024-03-10
8,112400-3197410-A-112,112400,ACTIVE,Briony,Hampton,Marylou,prefix,suffix,Female,null,A2,12,923 Boyle Dale Apt 44,null,Boston,Massachusetts,Suffolk County,02113,U.S.A,0,0,0,member@gmail.com,None,null,None,1,Abacus Insights,None,None,U,Unknown,10.0,Plan K,Medical and Vision,2020-01-01,2020-12-31,null,null,null,null,Aetna,Member Enrolment,2024-03-10
9,114100-2119918-A-114,114100,ACTIVE,Safa,Kelly,None,prefix,suffix,Female,null,G3,9,765 Russel Pathway Unit 82,null,Spencer,Massachusetts,Worcester County,01562,U.S.A,0,0,0,member@gmail.com,None,null,None,8,TechKraft Inc,None,None,F,Family,10.0,Plan K,Medical and Vision,2020-01-01,2020-12-31,null,nul